In [ ]:
import polars as pl
import numpy as np
from datetime import datetime, timezone, timedelta
from zoneinfo import ZoneInfo
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['timezone'] = 'Europe/Moscow'

import ast
import math
import pickle
from bot.utils.pair_trading import make_df_from_orderbooks, make_trunc_df, make_zscore_df
from bot.analysis.strategy_analysis import analyze_strategy, create_pair_trades_df
from bot.utils.coins import get_step_info, get_price_scale

from bot.db.postgres_manager import DBManager
from bot.config.credentials import host, user, password, db_name
db_params = {'host': host, 'user': user, 'password': password, 'dbname': db_name}
db_manager = DBManager(db_params)

from bot.utils.pair_trading import create_zscore_df, get_lr_zscore, get_dist_zscore, get_qty, round_down, calculate_profit_curve
from bot.utils.data import calculate_profit

with open("./data/coin_information.pkl", "rb") as f:
    coin_information = pickle.load(f)

In [19]:
trading_history = db_manager.get_table('trading_history', df_type='polars')

In [20]:
trading_history.sort(by='close_time').tail(3)

token_1,token_2,open_time,close_time,side_1,side_2,qty_1,qty_2,open_price_1,open_price_2,close_price_1,close_price_2,fee_1,fee_2,leverage,pnl_1,pnl_2,profit
str,str,"datetime[μs, Europe/Moscow]","datetime[μs, Europe/Moscow]",str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""ARKM""","""MANTA""",2025-11-07 15:56:16 MSK,2025-11-07 17:13:33 MSK,"""long""","""short""",336.0,937.9,0.2964,0.10658,0.3051,0.10487,-0.111157,-0.109075,2.0,2.8120428,1.494734,4.306776
"""LDO""","""SAND""",2025-11-07 00:49:13 MSK,2025-11-07 19:34:53 MSK,"""long""","""short""",135.8,553.0,0.7353,0.1807,0.8061,0.1998,-0.113001,-0.095834,2.0,9.501639,-10.658134,-1.156495
"""GALA""","""GMT""",2025-11-07 15:50:46 MSK,2025-11-07 21:01:17 MSK,"""short""","""long""",11229.0,4615.0,0.008895,0.02163,0.010292,0.02485,-0.128608,-0.128602,2.0,-15.815521,14.731698,-1.083822


In [21]:
trading_history['profit'].sum()

24.864112649999992

In [ ]:
trading_history.group_by('token_1', 'token_2').agg(
    pl.col('profit').sum().round(2),
    pl.col('profit').min().round(2).alias('min_profit'),
    pl.col('profit').max().round(2).alias('max_profit'),
    (pl.col('profit').sum() / pl.col('profit').len()).round(2).alias('avg_profit'),
    pl.col('profit').len().alias('n_trades'),

).sort(by='profit', descending=True)

token_1,token_2,profit,min_profit,max_profit,avg_profit,n_trades
str,str,f64,f64,f64,f64,u32
"""CVX""","""SUSHI""",7.81,0.19,4.4,1.95,4
"""ARKM""","""MANTA""",6.52,-0.83,4.31,1.63,4
"""CHZ""","""KAS""",6.3,-0.22,6.52,3.15,2
"""MORPHO""","""STX""",5.48,1.27,4.2,2.74,2
"""CRV""","""SUI""",4.28,1.76,2.53,2.14,2
…,…,…,…,…,…,…
"""GALA""","""GMT""",-0.05,-1.53,1.58,-0.01,5
"""LDO""","""SAND""",-0.76,-1.16,0.4,-0.38,2
"""IOTA""","""MOVE""",-3.02,-6.15,2.96,-1.01,3


In [ ]:
def create_report(token_1: str,
                  token_2: str,
                  open_time: datetime,
                  close_time: datetime,
                  spread_method: str,
                  min_order: int,
                  sl_ratio: float = None,
                  plot: bool=True):
    """
    Функция для анализа криптовалютной пары.

    Args:
        token_1, token_2: названия монет, указываются без суффикса '_USDT'
        open_time: время входа в сделку
        close_time: время выхода из сделки
        spread_method: метод, которым рассчитывается спред между монетами.
            'lr' для метода линейной регрессии,
            'dist' для дистанционного метода (разность логарифмов цен)
        min_order: размер минимального ордера в usdt. Используется для отфильтровывания цен с малым объёмом
        plot: нужно ли выводить график на экран

    """

    open_ts = int(datetime.timestamp(open_time))
    close_ts = int(datetime.timestamp(close_time))


    trading_history = db_manager.get_table('trading_history', df_type='polars')

    z_score_hist = db_manager.get_zscore_history(token_1 + '_USDT', token_2 + '_USDT', open_ts, close_ts)

    tf = '4h'
    wind = 24
    winds = np.array((wind, ))
    thresh_in = 2.25
    thresh_out = 0.25
    side = open_order_data['side'][0]
    t2_side = 'long' if side == 'short' else 'short'

    beta_open_order = open_order_data['beta'][0]

    t1_open_data = open_order_data['t1'][0]
    t2_open_data = open_order_data['t2'][0]
    t1_close_data = close_order_data['t1'][0]
    t2_close_data = close_order_data['t2'][0]

    t1_real_data = trading_history.filter(
            (pl.col('token') == token_1 + '_USDT') & (abs(pl.col('created_at') - open_time_order) < timedelta(seconds=10))
        )
    t2_real_data = trading_history.filter(
            (pl.col('token') == token_2 + '_USDT') & (abs(pl.col('created_at') - open_time_order) < timedelta(seconds=10))
        )
    open_time_real = max(t1_real_data['created_at'][0], t2_real_data['created_at'][0])
    close_time_real = max(t1_real_data['closed_at'][0], t2_real_data['closed_at'][0])
    leverage_1 = int(t1_real_data['leverage'][0])
    leverage_2 = int(t2_real_data['leverage'][0])

    assert leverage_1 == leverage_2

    start_ts = int(datetime.timestamp(open_time_order))
    median_length = 6

    # Реальный размер позиции
    t1_qty = t1_real_data['qty'][0]
    t2_qty = t2_real_data['qty'][0]

    t1_op_order = open_order_data['t1_ask_price'][0] if side == 'long' else open_order_data['t1_bid_price'][0]
    t2_op_order = open_order_data['t2_bid_price'][0] if side == 'long' else open_order_data['t2_ask_price'][0]
    t1_op_real = t1_real_data['open_price'][0]
    t2_op_real = t2_real_data['open_price'][0]

    t1_cp_order = close_order_data['t1_ask_price'][0] if side == 'short' else close_order_data['t1_bid_price'][0]
    t2_cp_order = close_order_data['t2_bid_price'][0] if side == 'short' else close_order_data['t2_ask_price'][0]
    t1_cp_real = t1_real_data['close_price'][0]
    t2_cp_real = t2_real_data['close_price'][0]

    t1_profit = t1_real_data['profit'][0]
    t2_profit = t2_real_data['profit'][0]
    total_profit = t1_profit + t2_profit

    # vol-neutral
    total_pos_side = round(t1_qty * t1_op_real + t2_qty * t2_op_real)
    t1 = open_order_data['t1'][0]
    t2 = open_order_data['t2'][0]
    std_1, std_2 = np.std(t1.to_list()), np.std(t2.to_list())

    t1_qty_vn, t2_qty_vn = get_qty(token_1, token_2, t1_op_real, t2_op_real, beta=None, coin_information=coin_information,
                        total_usdt_amount=total_pos_side, fee_rate=fee_rate, std_1=std_1, std_2=std_2, method='vol_neutral')

    t1_profit_vn = calculate_profit(t1_op_real, t1_cp_real, t1_qty_vn, side)
    t2_profit_vn = calculate_profit(t2_op_real, t2_cp_real, t2_qty_vn, t2_side)

    profit_vn = t1_profit_vn + t2_profit_vn

    # beta-neutral
    t1_qty_bn, t2_qty_bn = get_qty(token_1, token_2, t1_op_real, t2_op_real, beta=beta_open_order, coin_information=coin_information,
                        total_usdt_amount=total_pos_side, fee_rate=fee_rate, method='beta_neutral')
    t1_profit_bn = calculate_profit(t1_op_real, t1_cp_real, t1_qty_bn, side)
    t2_profit_bn = calculate_profit(t2_op_real, t2_cp_real, t2_qty_bn, t2_side)

    profit_bn = t1_profit_bn + t2_profit_bn

    # spread
    spread, spread_mean, spread_std, alpha, beta, zscore = get_lr_zscore(t1.to_numpy(), t2.to_numpy(), np.array([wind]))
    spread, spread_mean, spread_std, alpha, beta, zscore = spread[0], spread_mean[0], spread_std[0], alpha[0], beta[0], zscore[0]

    dist_mean, dist_std, dist_z = get_dist_zscore(t1.to_numpy(), t2.to_numpy(), np.array([wind]))
    dist_mean, dist_std, dist_z = float(dist_mean[0]), float(dist_std[0]), float(dist_z[0])
    dist_spread = dist_mean + dist_z * dist_std

    if plot:
        train_len = int(tf[0]) * wind * 2
        start_time = open_time_order - timedelta(hours = train_len)

        df_1 = db_manager.get_tick_ob(token=token_1 + '_USDT',
                                         start_time=start_time,
                                         end_time=close_time_real)
        df_2 = db_manager.get_tick_ob(token=token_2 + '_USDT',
                                         start_time=start_time,
                                         end_time=close_time_real)

        avg_df = make_df_from_orderbooks(df_1, df_2, token_1, token_2, start_time=start_time)

        if tf == '1h':
            agg_df = make_trunc_df(avg_df, timeframe='1h', token_1=token_1, token_2=token_2, method='triple')
        elif tf == '4h':
            agg_df = make_trunc_df(avg_df, timeframe='4h', token_1=token_1, token_2=token_2, method='triple', offset='3h')

        df_sec = make_df_from_orderbooks(df_1, df_2, token_1, token_2, start_time=start_time)
        spread_df = create_zscore_df(token_1, token_2, df_sec, agg_df, tf, winds, min_order, start_ts, median_length)

        df = spread_df.select('time', 'ts', token_1, token_2, f'{token_1}_size', f'{token_2}_size',
             f'{token_1}_bid_price', f'{token_1}_ask_price', f'{token_1}_bid_size', f'{token_1}_ask_size',
             f'{token_2}_bid_price', f'{token_2}_ask_price', f'{token_2}_bid_size', f'{token_2}_ask_size',
             f'z_score_{wind}_{tf}').filter(
                (pl.col('time') >= start_time) & (pl.col('time') <= close_time)
             ).rename({f'z_score_{wind}_{tf}': 'z_score'})

        if additional_ts:
            add_time = datetime.fromtimestamp(additional_ts)
            add_row = df.filter(pl.col('ts') == additional_ts)
            t1_add_close = add_row[f'{token_1}_ask_price'][0] if side == 'short' else add_row[f'{token_1}_bid_price'][0]
            t2_add_close = add_row[f'{token_2}_bid_price'][0] if side == 'short' else add_row[f'{token_2}_ask_price'][0]

            pr_1 = calculate_profit(open_price=t1_op_real, close_price=t1_add_close, n_coins=t1_qty, side=side)
            pr_2 = calculate_profit(open_price=t2_op_real, close_price=t2_add_close, n_coins=t2_qty, side=t2_side)

        sl_profit = None
        if sl_ratio:
            pos_size = (t1_qty * t1_op_real + t2_qty * t2_op_real) / leverage_1
            sl_idx = z_score_hist.filter(pl.col('profit') < -sl_ratio * pos_size).head(1)

            if sl_idx.height > 0:
                sl_row = df.filter((m:=(pl.col('ts') - sl_idx['ts'][0]).abs()).min() == m)
                sl_time = sl_row['time'][0]
                t1_sl_close = sl_row[f'{token_1}_ask_price'][0] if side == 'short' else sl_row[f'{token_1}_bid_price'][0]
                t2_sl_close = sl_row[f'{token_2}_bid_price'][0] if side == 'short' else sl_row[f'{token_2}_ask_price'][0]

                sl_pr_1 = calculate_profit(open_price=t1_op_real, close_price=t1_sl_close, n_coins=t1_qty, side=side)
                sl_pr_2 = calculate_profit(open_price=t2_op_real, close_price=t2_sl_close, n_coins=t2_qty, side=t2_side)
                sl_profit = sl_pr_1 + sl_pr_2

        coef = df[token_1].mean() / df[token_2].mean()
        fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(14, 6), sharex=True)

        # --- Графики цены ---
        ax1.plot(df.select('time'), df.select(token_1), label=token_1, color='orange')
        ax1.set_title(f'{token_1} - {token_2} (tf: {tf}, wind: {wind}, side: {side}; thresh_in: {thresh_in}, thresh_out: {thresh_out})')
        if additional_ts:
            ax1.axvline(x=add_time-timedelta(hours=3), color='black', linestyle='dotted')

        ax1.set_ylabel('udst')
        ax1.legend()
        ax1.grid()

        ax4 = ax1.twinx()
        ax4.plot(df.select('time'), df.select(token_2), label=token_2, color='blue')

        # --- Графики z_score ---
        ax2.plot(df.select('time'), df.select('z_score'), label=f'calculated_z_score')
        ax2.plot(z_score_hist.select('time'), z_score_hist.select('z_score'), label=f'real_z_score')
        ax2.set_ylabel('z_score')
        if additional_ts:
            ax2.axvline(x=add_time-timedelta(hours=3), color='black', linestyle='dotted')
        if side == 'long':
            ax2.axhline(-thresh_in, c='g', linestyle='dotted')
            ax2.axhline(thresh_out, c='r', linestyle='dotted')
        else:
            ax2.axhline(thresh_in, c='g', linestyle='dotted')
            ax2.axhline(-thresh_out, c='r', linestyle='dotted')
        ax2.grid()
        ax2.legend()

        # --- График профита ---
        ax3.plot(z_score_hist.select('time'), z_score_hist.select('profit'), color='green', label='profit')
        if additional_ts:
            ax3.axvline(x=add_time-timedelta(hours=3), color='black', linestyle='dotted')
        ax3.set_ylabel('profit')
        ax3.grid()
        ax3.legend()

        plt.tight_layout()
        plt.show()

    print(f'{token_1} - {token_2}. Timeframe: {tf}; window: {wind}; thresh_in: {thresh_in}, thresh_out: {thresh_out}')
    print(f'[ OPEN] Время создания ордера: {open_order_data['ct'][0]:%Y-%m-%d %H:%M:%S}, время исполнения ордера: {open_time_real:%Y-%m-%d %H:%M:%S}')
    print(f'Цены во время открытия. {token_1}: order: {t1_op_order}, real: {t1_op_real}; {token_2}: order: {t2_op_order}, real: {t2_op_real}')
    print(f'z_score во время создания ордера: {open_order_data['z_score'][0]}, на момент исполнения: {z_score_hist['z_score'][0]:.2f}')
    print(f'Волатильность в момент открытия: {token_1}: {std_1:.4f}; {token_2}: {std_2:.4f}')
    print(f'Линейная регрессия.    spread: {spread:.5f}, spread_mean: {spread_mean:.5f}, spread_std: {spread_std:.5f}, \
alpha: {alpha:.2f}, beta: {beta:.2f}, z_score: {zscore:.2f}')
    print(f'Логарифмический спред. spread: {dist_spread:.5f}, spread_mean: {dist_mean:.5f}, spread_std: {dist_std:.5f}, z_score: {dist_z:.2f}')
    print()
    print(f'[CLOSE] Время создания ордера: {close_order_data['ct'][0]:%Y-%m-%d %H:%M:%S}, время исполнения ордера: {close_time_real:%Y-%m-%d %H:%M:%S}')
    print(f'Цены во время закрытия. {token_1}: order: {t1_cp_order}, real: {t1_cp_real}; {token_2}: order: {t2_cp_order}, real: {t2_cp_real}')
    print(f'z_score во время создания ордера: {close_order_data['z_score'][0]}, на момент исполнения: {z_score_hist['z_score'][-1]:.2f}')

    print()
    print(f'[PROFIT]')
    print(f'usdt-neutral: {t1_qty} {token_1}({side}): {t1_profit:.2f}$, {t2_qty} {token_2}({t2_side}): {t2_profit:.2f}$; total profit: {total_profit:.2f}$')
    print(f'vol-neutral : {t1_qty_vn} {token_1}({side}): {t1_profit_vn:.2f}$, {t2_qty_vn} {token_2}({t2_side}): {t2_profit_vn:.2f}$; total profit: {profit_vn:.2f}$')
    print(f'beta-neutral: {t1_qty_bn} {token_1}({side}): {t1_profit_bn:.2f}$, {t2_qty_bn} {token_2}({t2_side}): {t2_profit_bn:.2f}$; total profit: {profit_bn:.2f}$')

    if additional_ts:
        print(f'\nДополнительная отсечка. Цены. {token_1}: {t1_add_close}; {token_2}: {t2_add_close}')
        print(f'Profit: {token_1}({side}): {pr_1:.2f}$, {token_2}({t2_side}): {pr_2:.2f}$; total profit: {pr_1 + pr_2:.2f}$')

    if sl_profit:
        print(f'\n[STOP LOSS] Время создания ордера: {sl_time:%Y-%m-%d %H:%M:%S}')
        print(f'Stop-loss. Цены. {token_1}: {t1_sl_close}; {token_2}: {t2_sl_close}')
        print(f'Profit: {token_1}({side}): {sl_pr_1:.2f}$, {token_2}({t2_side}): {sl_pr_2:.2f}$; total profit: {sl_profit:.2f}$')

    if sl_profit:
        return sl_profit
    else:
        return total_profit, profit_vn, profit_bn

In [ ]:
token_1 = 'AI16Z'
token_2 = 'CELO'
start_time = "2025-10-03 06:32:48"
end_time = "2025-10-04 01:02:16"
additional_ts = int(datetime.timestamp(datetime(2025, 9, 16, 20, 20)))

open_time = datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S').replace(tzinfo=ZoneInfo("Europe/Moscow"))
close_time = datetime.strptime(end_time, '%Y-%m-%d %H:%M:%S').replace(tzinfo=ZoneInfo("Europe/Moscow"))

profit, profit_vn, profit_bn = create_report(token_1, token_2, open_time, close_time, spread_method='lr',
              # additional_ts=additional_ts,
              sl_ratio=0.99,
              min_order=50)